Fine tuning not making from scratch, coz faster

We can also make from scratch if we have a big enough data

In [1]:
%%capture
!pip install datasets

In [3]:
import pandas as pd
df = pd.read_csv('/content/shakespearify.csv')


In [4]:
df.head()

,Unnamed: 0,id,og,t
0,0,42928-1500614319216-63344,You do not meet a man but frowns:,Every man you meet these days is frowning.
1,1,42928-1500614326583-89821,our bloods No more obey the heavens than our...,Our bodies are in agreement with the planetar...
2,2,A-63849,But what's the matter?,What's wrong?
3,3,42930-1500614347266-80123,"His daughter, and the heir of's kingdom, whom...","The king wanted his daughter, the only heir to..."
4,4,42930-1500614355280-38326,she's wedded; Her husband banish'd; she impr...,"She's married, her husband is banished, she's..."


In [5]:
df.drop(df.columns[0], inplace = True, axis = 1)
df.head()

,id,og,t
0,42928-1500614319216-63344,You do not meet a man but frowns:,Every man you meet these days is frowning.
1,42928-1500614326583-89821,our bloods No more obey the heavens than our...,Our bodies are in agreement with the planetar...
2,A-63849,But what's the matter?,What's wrong?
3,42930-1500614347266-80123,"His daughter, and the heir of's kingdom, whom...","The king wanted his daughter, the only heir to..."
4,42930-1500614355280-38326,she's wedded; Her husband banish'd; she impr...,"She's married, her husband is banished, she's..."


In [6]:
df.shape

(51787, 3)

In [7]:
from datasets import load_dataset, Dataset

ds = Dataset.from_pandas(df)

In [8]:
ds = ds.train_test_split(test_size = 0.2)

In [9]:
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'og', 't'],
        num_rows: 41429
    })
    test: Dataset({
        features: ['id', 'og', 't'],
        num_rows: 10358
    })
})

In [10]:
ds['train'][0:2]

{'id': ['54858-1503967629166-95023', '44707-1501503088404-54132'],
 'og': [' O, calm thee, gentle lord; although I know  There is enough written upon this earth  To stir a mutiny in the mildest thoughts  And arm the minds of infants to exclaims. ',
  " 'If once he come to be a cardinal,  He'll make his cap co-equal with the crown.' "],
 't': ['Oh, calm yourself, gentle lord—although I know there’s enough written here on the ground to enrage even the mildest person, and make children exclaim with anger. ',
  ' "If he becomes a cardinal one day, he\'ll make his Cardinal\'s hat as powerful as the crown."']}

In [11]:
from transformers import MarianTokenizer, MarianMTModel

model_checkpoint = "Helsinki-NLP/opus-mt-en-ROMANCE"
src_lang = "en"
tgt_lang = "shakespeare"
tokenizer = MarianTokenizer.from_pretrained(model_checkpoint, src_lang = src_lang, tgt_lang = tgt_lang)
model_ex = MarianMTModel.from_pretrained(model_checkpoint)



# Example function to translate text
def translate_to_shakespearean_english(text):
    # Tokenize the input text
    tokenized_text = tokenizer(text, return_tensors="pt", src_lang=src_lang)

    # Generate translation
    translated_tokens = model_ex.generate(**tokenized_text)

    # Decode the tokens back to text
    translated_text = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)

    return translated_text

# Example usage
text = "To be or not to be, that is the question."
shakespearean_translation = translate_to_shakespearean_english(text)
print(shakespearean_translation)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/265 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/779k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/799k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.46M [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

Keyword arguments {'src_lang': 'en'} not recognized.


Ser ou no ser, c'est la question.


In [ ]:
#Well, that is french coz marian mt doesn't have target language "shakespeare",
#So we will introuduce it then
#(IT won't work well coz we have so little data to train on, but let's see)

In [12]:
en_sen = "translate English to Shakespearean:" + ds['train']['t'][1]
sh_sen = '<shakespeare>' + ds['train']['og'][1]

inputs = tokenizer(en_sen, text_target = sh_sen)
print(inputs)

{'input_ids': [22612, 1342, 12, 48510, 440, 37, 58, 9932, 128, 6590, 9, 30028, 143, 515, 2, 128, 7, 240, 342, 177, 25737, 7, 6, 16696, 53, 6963, 53, 4, 19772, 403, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [1284, 8685, 1565, 4868, 954, 11862, 61, 404, 481, 49, 1101, 128, 295, 12, 48, 9, 30028, 2, 233, 7, 240, 343, 1565, 4246, 6, 3230, 486, 15, 43, 15514, 44, 4, 5922, 10630, 3, 7, 0]}


In [13]:
# I got the input ids and labels to train em on,

In [14]:
tokenizer.decode(inputs.labels)

"<shakespeare> 'If once he come to be a cardinal, He'll make his cap co-equal with the crown.'</s>"

In [15]:
max_length = 128

def preprocess_function(examples):
    inputs = [f"translate English to Shakespearean: {ex}" for ex in examples["t"]]  # 't' contains shakespearen
    targets = [f"<shakespeare> {ex}" for ex in examples["og"]]  # 'og' contains English

    # Tokenize the input and target sentences
    model_inputs = tokenizer(inputs, max_length=max_length, truncation=True)
    labels = tokenizer(targets, max_length=max_length, truncation=True)
    # Prepare labels for the model
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs



In [16]:
tokenized_ds = ds.map(preprocess_function, batched=True, remove_columns = ds['train'].column_names )
print(tokenized_ds)

Map:   0%|          | 0/41429 [00:00<?, ? examples/s]

Map:   0%|          | 0/10358 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 41429
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 10358
    })
})


In [17]:
ds['train'][0]['og']

' O, calm thee, gentle lord; although I know  There is enough written upon this earth  To stir a mutiny in the mildest thoughts  And arm the minds of infants to exclaims. '

In [ ]:
tokenizer.decode(tokenized_ds['train'][0]['labels'])

'<shakespeare> Good morrow to you both. What counterfeit did I give you?</s>'

In [ ]:
from transformers import TFAutoModelForSeq2SeqLM

model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint, from_pt=True)

All PyTorch model weights were used when initializing TFMarianMTModel.

All the weights of TFMarianMTModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

In [ ]:
batch = data_collator([tokenized_ds["train"][i] for i in range(1, 3)])
batch.keys()

dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])

In [ ]:
tf_train_dataset = model.prepare_tf_dataset(
    tokenized_ds["train"],
    collate_fn=data_collator,
    shuffle=True,
    batch_size=32,
)
tf_eval_dataset = model.prepare_tf_dataset(
    tokenized_ds["test"],
    collate_fn=data_collator,
    shuffle=False,
    batch_size=16,
)

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import create_optimizer
from transformers.keras_callbacks import PushToHubCallback
import tensorflow as tf

# The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
# by the total number of epochs. Note that the tf_train_dataset here is a batched tf.data.Dataset,
# not the original Hugging Face Dataset, so its len() is already num_samples // batch_size.
num_epochs = 3
num_train_steps = len(tf_train_dataset) * num_epochs

optimizer, schedule = create_optimizer(
    init_lr=5e-5,
    num_warmup_steps=1000,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)
model.compile(optimizer=optimizer)

# Train in mixed-precision float16
tf.keras.mixed_precision.set_global_policy("mixed_float16")

In [ ]:
from transformers.keras_callbacks import PushToHubCallback

callback = PushToHubCallback(
    output_dir="marian-finetuned-english-to-shakesperian", tokenizer=tokenizer
)

model.fit(
    tf_train_dataset,
    validation_data=tf_eval_dataset,
    callbacks=[callback],
    epochs=1,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/Sanyam52/marian-finetuned-english-to-shakesperian into local empty directory.


Epoch 1/3
1294/1294 [==============================] - ETA: 0s - loss: 3.4431

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[65000]], 'forced_eos_token_id': 0}
Adding files tracked by Git LFS: ['source.spm', 'target.spm']. This may take a bit of time if the files are large.


1294/1294 [==============================] - 775s 572ms/step - loss: 3.4431 - val_loss: 2.4710
Epoch 2/3
1294/1294 [==============================] - ETA: 0s - loss: 2.3504

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[65000]], 'forced_eos_token_id': 0}


1294/1294 [==============================] - 771s 596ms/step - loss: 2.3504 - val_loss: 2.2042
Epoch 3/3
1294/1294 [==============================] - ETA: 0s - loss: 2.1085

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[65000]], 'forced_eos_token_id': 0}


1294/1294 [==============================] - 776s 600ms/step - loss: 2.1085 - val_loss: 2.1420


In [ ]:

def translate_to_shakespearean(text):
    # Tokenize and generate translation
    # Change return_tensors to "tf"
    tokenized_text = tokenizer(text, return_tensors="tf")
    translated_tokens = model.generate(**tokenized_text)

    # Decode the output tokens into text
    return tokenizer.decode(translated_tokens[0], skip_special_tokens=True)

# Test examples
modern_english_texts = [
    "Where are you going?",
    "To be or not to be, that is the question.",
    "Hello, how are you?",
    "I love you.",
    "The night is dark and full of terrors."
]

for text in modern_english_texts:
    shakespearean_translation = translate_to_shakespearean(text)
    print(f"Modern English: {text}")
    print(f"Shakespearean English: {shakespearean_translation}\n")


Modern English: Where are you going?
Shakespearean English: Where you go?

Modern English: To be or not to be, that is the question.
Shakespearean English: To be or no, that is the question.

Modern English: Hello, how are you?
Shakespearean English: How now?

Modern English: I love you.
Shakespearean English: I love thee.

Modern English: The night is dark and full of terrors.
Shakespearean English: The night is dark and full of terrors.



In [ ]:
# The output is somehting, its not perfect but at least it got I love thee right, let's compute the bleu metric and then train it some more

1294/1294 [==============================] - ETA: 0s - loss: 2.0346

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[65000]], 'forced_eos_token_id': 0}


1294/1294 [==============================] - 758s 586ms/step - loss: 2.0346 - val_loss: 2.1420


In [ ]:
#Trained it for one more epoch, which didn't affect the val loss at all. So I need to reduce the learning rate even more or just get more data
#Let's start by computing the metrics and then we will se what to do with the learning rate and training

In [ ]:
# BLEU Metric
"""
  The score can go from 0 to 100, and higher is better.

"""

'\n  The score can go from 0 to 100, and higher is better.\n\n'

In [ ]:
%%capture
!pip install sacrebleu
!pip install evaluate

In [ ]:
import evaluate
metric = evaluate.load("sacrebleu")

In [ ]:
predictions = [
    "This plugin lets you translate web pages between several languages automatically."
]
references = [
    [
        "This plugin allows you to automatically translate web pages between several languages."
    ]
]
metric.compute(predictions=predictions, references=references)

{'score': 46.750469682990165,
 'counts': [11, 6, 4, 3],
 'totals': [12, 11, 10, 9],
 'precisions': [91.66666666666667,
  54.54545454545455,
  40.0,
  33.333333333333336],
 'bp': 0.9200444146293233,
 'sys_len': 12,
 'ref_len': 13}

In [24]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, MarianMTModel

tokenizer = AutoTokenizer.from_pretrained("Sanyam52/marian-finetuned-english-to-shakesperian")
# Use MarianMTModel and from_tf=True to load the TensorFlow model
model = TFAutoModelForSeq2SeqLM.from_pretrained("Sanyam52/marian-finetuned-english-to-shakesperian")

All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Sanyam52/marian-finetuned-english-to-shakesperian.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


In [25]:
import numpy as np
import tensorflow as tf
from tqdm import tqdm
from transformers import DataCollatorForSeq2Seq

generation_data_collator = DataCollatorForSeq2Seq(
    tokenizer, model=model, return_tensors="tf", pad_to_multiple_of=128
)

tf_generate_dataset = model.prepare_tf_dataset(
    tokenized_ds["test"],
    collate_fn=generation_data_collator,
    shuffle=False,
    batch_size=8,
)


def generate_with_xla(batch):
  return model.generate(
      input_ids=batch["input_ids"],
      attention_mask=batch["attention_mask"],
      max_new_tokens=128,
  )

def compute_metrics():
  all_preds = []
  all_labels = []

  for batch, labels in tqdm(tf_generate_dataset):
    predictions = generate_with_xla(batch)
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = labels.numpy()
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    all_preds.extend(decoded_preds)
    all_labels.extend(decoded_labels)

  result = metric.compute(predictions=all_preds, references=all_labels)
  return {"bleu": result["score"]}

In [ ]:
print(compute_metrics())
#Don't have the computational power to run this whole test

  1%|          | 10/1295 [05:56<11:16:17, 31.58s/it]